In [15]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import re
from Constants import *
import simplejson
from Preprocessing import *
from Models import *
from DeepSurvivalModels import *
from Utils import load_models
from SymptomPrediction import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
decision_model,transition_model1,transition_model2, outcome_model,survival_model = load_models()
survival_model.k

6

In [17]:
sp, mdasi = load_mdasi_stuff()
sp

SymptomPredictor(
  (layers): ModuleList(
    (0): Linear(in_features=27, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=100, bias=True)
    (3): ReLU()
  )
  (batchnorm): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (relu): ReLU()
  (sigmoid): Sigmoid()
  (final_layer): Linear(in_features=100, out_features=112, bias=True)
)

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
decision_model.set_device(device)
transition_model1.set_device(device)
transition_model2.set_device(device)
outcome_model.set_device(device)
survival_model.set_device(device)

In [20]:
data = DTDataset()
newdf = data.processed_df.copy()
for c in newdf.columns:
    if newdf[c].dtype == np.float64:
        newdf[c] = newdf[c].astype(np.float32).apply(lambda x: np.round(x,2))
data.processed_df = newdf
data.processed_df

,hpv,age,packs_per_year,gender,Aspiration rate Pre-therapy,total_dose,dose_fraction,OS (Calculated),Locoregional control (Time),FDM (months),...,4_ipsi,4_contra,5A_ipsi,5A_contra,5B_ipsi,5B_contra,6_ipsi,6_contra,RPLN_ipsi,RPLN_contra
id,,,,,,,,,,,,,,,,,,,,,
3,1,55.97,0.0,1,0,66.00,2.20,6.03,4.70,6.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,20.95,38.0,1,0,72.00,1.80,7.33,7.33,7.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,69.93,35.0,0,1,70.00,2.12,7.47,7.47,7.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1,72.32,0.0,1,0,70.00,2.12,7.80,7.80,7.80,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1,59.73,0.0,1,0,66.00,2.20,8.07,8.07,8.07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10201,1,49.57,30.0,1,0,70.00,2.12,143.20,143.20,143.20,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10202,0,48.71,30.0,1,0,72.00,1.71,144.37,144.37,144.37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10203,1,77.12,0.0,1,0,70.00,2.33,148.37,148.37,136.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
def df_to_torch(df,ttype  = torch.FloatTensor):
    values = df.values.astype(float)
    values = torch.from_numpy(values)
    return values.type(ttype)

def get_decision_input(dataset,state=0,ids=None):
    baseline = dataset.get_state('baseline')
    dlt1 = dataset.get_state('dlt1')
    dlt2 = dataset.get_state('dlt2')
    pd1 = dataset.get_state('pd_states1')
    pd2 = dataset.get_state('pd_states2')
    nd1 = dataset.get_state('nd_states1')
    nd2 = dataset.get_state('nd_states2')
    modifications = dataset.get_state('modifications')
    ccs = dataset.get_state('ccs')
    if state < 2:
        pd = pd1.copy()
        nd = nd1.copy()
        dlt2.values[:,:] = np.zeros(dlt2.shape)
        ccs.values[:,:] = np.zeros(ccs.shape)
        if state < 1:
            dlt1.values[:,:] = np.zeros(dlt1.shape)
            modifications.values[:,:] = np.zeros(modifications.shape)
            pd.values[:,:] = np.zeros(pd.shape)
            nd.values[:,:] = np.zeros(nd.shape)
    else:
        pd = pd2.copy()
        nd = nd2.copy()
        
    output = [baseline, dlt1, dlt2, pd, nd,ccs,modifications]
    if ids is not None:
        output = [o.loc[ids] for o in output]
    return output
test_patient_id=7
get_decision_input(data)[0]

,1A_contra,1A_ipsi,1B_contra,1B_ipsi,2A_contra,2A_ipsi,2B_contra,2B_ipsi,3_contra,3_ipsi,...,dose_fraction,gender,hpv,packs_per_year,subsite_BOT,subsite_GPS,subsite_NOS,subsite_Soft palate,subsite_Tonsil,total_dose
id,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,2.20,1,1,0.0,1,0,0,0,0,66.00
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.80,1,0,38.0,1,0,0,0,0,72.00
6,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.12,0,1,35.0,1,0,0,0,0,70.00
7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.12,1,1,0.0,0,0,1,0,0,70.00
8,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,2.20,1,1,0.0,0,0,0,0,1,66.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10201,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,2.12,1,1,30.0,1,0,0,0,0,70.00
10202,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.71,1,0,30.0,0,0,1,0,0,72.00
10203,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,2.33,1,1,0.0,0,0,0,0,1,70.00


In [22]:
def get_inputkey_order(dataset,state=0):
    return [list(f.columns) for f in get_decision_input(dataset,state=state)]

get_inputkey_order(data,1)

[['1A_contra',
  '1A_ipsi',
  '1B_contra',
  '1B_ipsi',
  '2A_contra',
  '2A_ipsi',
  '2B_contra',
  '2B_ipsi',
  '3_contra',
  '3_ipsi',
  '4_contra',
  '4_ipsi',
  '5A_contra',
  '5A_ipsi',
  '5B_contra',
  '5B_ipsi',
  '6_contra',
  '6_ipsi',
  'AJCC_1',
  'AJCC_2',
  'AJCC_3',
  'AJCC_4',
  'African American/Black',
  'Asian',
  'Aspiration rate Pre-therapy',
  'DLT_Infection (Pneumonia)',
  'DLT_Infection (Pneumonia) 2',
  'DLT_Nephrological',
  'DLT_Nephrological 2',
  'DLT_Vascular',
  'DLT_Vascular 2',
  'Hispanic/Latino',
  'N-category_0',
  'N-category_1',
  'N-category_2',
  'N-category_3',
  'Pathological Grade_0',
  'Pathological Grade_1',
  'Pathological Grade_2',
  'Pathological Grade_3',
  'Pathological Grade_4',
  'RPLN_contra',
  'RPLN_ipsi',
  'T-category_1',
  'T-category_2',
  'T-category_3',
  'T-category_4',
  'White/Caucasion',
  'age',
  'bilateral',
  'dose_fraction',
  'gender',
  'hpv',
  'packs_per_year',
  'subsite_BOT',
  'subsite_GPS',
  'subsite_NOS',
 

In [23]:
def get_test_patient(d,pid=7,clear_transitions=True):
    tp = d.processed_df.loc[pid].to_dict()
    if clear_transitions:
        for v in Const.primary_disease_states + Const.nodal_disease_states:
            tp[v] = 0
            tp[v + ' 2'] = 0
    return tp 
test_patient = get_test_patient(data)
test_patient

{'hpv': 1,
 'age': 72.32,
 'packs_per_year': 0.0,
 'gender': 1,
 'Aspiration rate Pre-therapy': 0,
 'total_dose': 70.0,
 'dose_fraction': 2.12,
 'OS (Calculated)': 7.8,
 'Locoregional control (Time)': 7.8,
 'FDM (months)': 7.8,
 'time_to_event': 6.0,
 'Overall Survival (1=alive, 0=dead)': 0,
 'LRC': 1,
 'DC': 1,
 'bilateral': False,
 'White/Caucasion': True,
 'Hispanic/Latino': False,
 'African American/Black': False,
 'Asian': False,
 'cc_none': 1,
 'cc_platinum': 0,
 'cc_cetuximab': 0,
 'cc_others': 0,
 'no_dose_adjustment': 1,
 'dose_modified': 0,
 'dose_delayed': 0,
 'dose_cancelled': 0,
 'dose_delayed_&_modified': 0,
 'regiment_modification': 0,
 'T-category_1': 1,
 'T-category_2': 0,
 'T-category_3': 0,
 'T-category_4': 0,
 'N-category_0': 0,
 'N-category_1': 0,
 'N-category_2': 1,
 'N-category_3': 0,
 'AJCC_1': 0,
 'AJCC_2': 1,
 'AJCC_3': 0,
 'AJCC_4': 0,
 'Pathological Grade_0': 1,
 'Pathological Grade_1': 0,
 'Pathological Grade_2': 0,
 'Pathological Grade_3': 0,
 'Pathologica

In [24]:
#from ClusterPredictions.py
test_patient = get_test_patient(data,7)
get_knn_predictions(test_patient,sp,mdasi,symptom_subset=['choke','drymouth'])

{'ids': [1237, 418, 1313, 1673, 361, 277, 298, 1483],
 'dists': [47.4503453343392,
  48.925031296142905,
  49.16342763346124,
  49.2733456894991,
  49.29173611348111,
  49.47347895424199,
  49.47525944749981,
  49.489565203565945],
 'symptoms': {'choke': {'ratings': [[0.0, 0.0, 0.0, -1.0],
    [0.0, -1.0, 1.0, 2.0],
    [3.0, 0.0, 0.0, -1.0],
    [-1.0, 1.0, -1.0, -1.0],
    [0.0, -1.0, -1.0, 0.0],
    [0.0, -1.0, 0.0, 0.0],
    [0.0, 1.0, 1.0, 4.0],
    [-1.0, 0.0, -1.0, 3.0]],
   'means': [0.5, 0.4, 0.4, 1.8]},
  'drymouth': {'ratings': [[0.0, 5.0, 2.0, -1.0],
    [0.0, -1.0, 6.0, 3.0],
    [3.0, 5.0, 3.0, -1.0],
    [-1.0, 1.0, -1.0, -1.0],
    [0.0, -1.0, -1.0, 0.0],
    [0.0, -1.0, 5.0, 3.0],
    [0.0, 8.0, 7.0, 6.0],
    [-1.0, 1.0, -1.0, 2.0]],
   'means': [0.5, 4.0, 4.6, 2.8]}}}

In [25]:
def get_embeddings(dataset,dm,states=[0,1,2],use_saved_memory=True,decimals=2):
    embeddings = []
    inputs = []
    decisions_optimal = [[] for i in states]
    decisions_imitation = [[] for i in states]
    for i,state in enumerate(states):
        x = get_decision_input(dataset,state=state)
        x = torch.cat([df_to_torch(f) for f in x],axis=1).to(dm.get_device())
        print(x.shape)
        embedding = dm.get_embedding(x,position = state,use_saved_memory=True)
        inputs.append(x.cpu().detach().numpy())
        decision = dm(x,position=state).cpu().detach().numpy()
        decisions_optimal[i].append(decision[:,state])
        decisions_imitation[i].append(decision[:,state+3])
        embedding = embedding.cpu().detach().numpy()
        if decimals is not None:
            embedding = np.round(embedding,decimals)
        embeddings.append(embedding)
    return embeddings,np.array(decisions_optimal).reshape(len(states),-1).T, np.array(decisions_imitation).reshape(len(states),-1).T, inputs
embeddings, decisions_optimal, decisions_imitation, testinputs = get_embeddings(data,decision_model)
[e.shape for e in embeddings]

torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])


[(536, 1000), (536, 1000), (536, 1000)]

In [26]:
#changed
def get_predictions(dataset,m1,m2,m3,sm3,states=[0,1,2],ids=None):
    outcomes = {}
    def add_outcomes(names, array,suffix=''):
        for i,name in enumerate(names):
            outcomes[name+suffix] = array[:,i]
    for model,state in zip([m1,m2,m3],states):
        x = dataset.get_input_state(step=state+1,ids=ids)
        x = df_to_torch(x).to(model.get_device())
        yout = model(x)
        y = yout['predictions']
        y_lower = yout['5%']
        y_upper = yout['95%']
        if state < 2:
            y = [yy.cpu().detach().numpy() for yy in y]
            y_lower = [yy.cpu().detach().numpy() for yy in y_lower]
            y_upper = [yy.cpu().detach().numpy() for yy in y_upper]
        else:
            y = y.cpu().detach().numpy()
            y_lower = y_lower.cpu().detach().numpy()
            y_upper = y_upper.cpu().detach().numpy()
        if state == 0:
            for suffixes, values in zip(['','_5%','_95%'],[y,y_lower,y_upper]):
                [pds, nd, mod, dlts] = values
                add_outcomes(Const.primary_disease_states,pds,suffixes)
                add_outcomes(Const.nodal_disease_states,nd,suffixes)
                add_outcomes(Const.modifications,mod,suffixes)
                add_outcomes(Const.dlt1,dlts,suffixes)
        elif state == 1:
            for suffixes, values in zip(['','_5%','_95%'],[y,y_lower,y_upper]):
                [pd2, nd2, cc, dlts2] = values
                add_outcomes(Const.primary_disease_states2,pd2,suffixes)
                add_outcomes(Const.nodal_disease_states2,nd2,suffixes)
                add_outcomes(Const.dlt2,dlts2,suffixes)
        else:
            for suffixes, values in zip(['','_5%','_95%'],[y,y_lower,y_upper]):
                add_outcomes(Const.outcomes,values,suffixes)
        if state == 2: #timeseries outcomes
            survival = sm3.time_to_event(x,n_samples=20)
            s = survival['predictions']
            s_lower = survival['5%']
            s_upper= survival['95%']
            names = survival['order']
            for suffixes,values in zip(['','_5%','_95%'],[s,s_lower,s_upper]):
                values = torch.stack(values,axis=1).cpu().detach().numpy()
                add_outcomes(names,values,suffixes)
    if ids is None:
        ids = dataset.processed_df.index.values
    outcomes = pd.DataFrame(outcomes,ids)
    outcomes.index.name = 'id'
    return outcomes
prediction_df = get_predictions(data,transition_model1,transition_model2,outcome_model,survival_model,ids=[3,5,7])
prediction_df[Const.timeseries_outcomes + [c+'_5%' for c in Const.timeseries_outcomes]]

,OS (Calculated),Locoregional control (Time),FDM (months),time_to_event,OS (Calculated)_5%,Locoregional control (Time)_5%,FDM (months)_5%,time_to_event_5%
id,,,,,,,,
3,276.478271,1060.589966,2004.874512,49.507156,254.452026,1014.422913,1684.298340,44.068024
5,245.843048,911.368835,1745.499512,34.479568,230.089935,801.499817,1568.682617,27.238298
7,329.097778,1238.492188,2055.133057,48.247662,303.553589,1084.653320,1965.405884,42.210831


In [27]:
from sklearn.decomposition import PCA
def get_embedding_pcas(dataset,decision_model,embeddings=None,components=2):
    if embeddings is None:
        embeddings, _, _, _ = get_embeddings(dataset,decision_model,states=[0,1,2])
    pcas = [PCA(components,whiten=True).fit(e) for e in embeddings]
    return pcas

def get_embedding_df(dataset,dm,states=[0,1,2],pcas=None,**kwargs):
    embeddings, decisions_opt, decisions_im, embedding_inputs = get_embeddings(dataset,dm,
                                                                                      states=states,**kwargs)
    values = {'embeddings_state'+str(i): [np.array(ee) for ee in e] for i,e in zip(states,embeddings)}
    newdf = pd.DataFrame(values,index=dataset.processed_df.index.values)
    for ii in states:
        opt = decisions_opt[:,ii]
        im = decisions_im[:,ii]
        newdf['decision'+str(ii)+"_optimal"] = opt
        newdf['decision'+str(ii)+'_imitation'] = im
        newdf['inputs'+str(ii)] = [np.array(ee) for ee in embedding_inputs[ii]]
    
    if pcas is None:
        pcas = get_embedding_pcas(dataset,dm,embeddings=embeddings)
    reductions = [ipca.fit_transform(e) for ipca,e in zip(pcas,embeddings)]
    for state,r in enumerate(reductions):
        newdf['pca_state'+str(state)] = [np.array(rr) for rr in r]
    return newdf

embedding_df = get_embedding_df(data,decision_model)
embedding_df

torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])


,embeddings_state0,embeddings_state1,embeddings_state2,decision0_optimal,decision0_imitation,inputs0,decision1_optimal,decision1_imitation,inputs1,decision2_optimal,decision2_imitation,inputs2,pca_state0,pca_state1,pca_state2
3,"[0.28, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.28, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.316692,0.975234,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",0.014232,0.046543,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...",0.238878,0.009288,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[-0.48599857, -0.506752]","[-0.45917124, -0.4639411]","[0.49081385, -0.4692875]"
5,"[0.27, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.26, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.206143,0.431012,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.002766,0.740672,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.005224,0.286323,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[1.2279854, -0.26386157]","[1.0977091, -0.046363775]","[-1.0250679, 0.3060964]"
6,"[0.28, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.152556,0.780169,"[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.001055,0.552226,"[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.001892,0.817031,"[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.6808392, -0.6142941]","[1.5127486, -0.60941494]","[-1.4664419, -0.11814983]"
7,"[0.55, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.52, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.47, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.382589,0.974404,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.005392,0.038135,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.007603,0.167392,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.22417444, -0.59941596]","[0.31850857, -0.85750437]","[-0.6068724, -0.6818945]"
8,"[0.39, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.22, 0.0...","[0.41, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.17, 0.0...","[0.34, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.42, 0.0...",0.646277,0.926530,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.268609,0.013533,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.827465,0.001125,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[-1.7481053, 0.70407444]","[-1.7431976, 0.8784005]","[2.068149, 0.6000858]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10201,"[0.34, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.33, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.39, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.343991,0.961293,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.004282,0.040797,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.028383,0.076898,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[0.1314283, -0.6250571]","[0.24043086, -0.6651105]","[-0.37348026, -0.41426834]"
10202,"[0.52, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.55, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.61, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.231363,0.662527,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.006583,0.582402,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.019196,0.064902,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[0.6399169, -0.50798887]","[0.61837995, -0.21105418]","[-0.59265363, 0.016543038]"
10203,"[0.23, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.22, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.597669,0.947900,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.013465,0.080935,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...",0.036082,0.026800,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, ...","[0.05236329, 0.14106381]","[0.053728197, 0.05760461]","[-0.108006194, 0.17732997]"
10204,"[0.51, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.78, 0.0...","[0.55, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.72

In [28]:
import simplejson
import pickle
def np_converter(obj):
    #converts stuff to vanilla python  for json since it gives an error with np.int64 and arrays
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.float32):
        return np.round(float(obj),3)
    elif isinstance(obj, float):
        return round(float(obj),3)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, datetime.datetime) or isinstance(obj, datetime.time):
        return obj.__str__()
    print('np_converter cant encode obj of type', obj,type(obj))
    return obj

def jsonify_np_dict(d):
    return simplejson.dumps(d,default=np_converter)


def get_dataset_jsons(dataset,ids=None,fields=None):
    df = dataset.processed_df.copy()
    print(ids,fields)
    if ids is not None and len(ids) > 0:
        df = df.loc[ids]
    if fields is not None and len(fields) > 0:
        df = df[fields]
    pdict = df.to_dict(orient='index')
    return jsonify_np_dict(pdict)


get_dataset_jsons(data,fields=['hpv','age'],ids=[3,7])

[3, 7] ['hpv', 'age']


'{"3": {"hpv": 1, "age": 55.97}, "7": {"hpv": 1, "age": 72.32}}'

In [29]:
def get_embedding_json(dataset,decisionmodel,embed_df = None,precision=4,ids=None,fields=None):
    if embed_df is None:
        embed_df = get_embedding_df(dataset,decisionmodel)
    if ids is not None and len(ids) > 0:
        embed_df = embed_df.loc[ids]
        
    for c in embed_df.columns:
        if 'embed' in c:
            embed_df[c] = embed_df[c].apply(lambda x: [round(float(xx),precision) for xx in x.astype(float)])
    to_keep = [c for c in embed_df if 'input' not in c]
    if fields is not None and len(fields) > 0:
        to_keep = [k for k in to_keep if k in fields]
    
    edict = embed_df[to_keep].to_dict(orient='index')
    return jsonify_np_dict(edict)

get_embedding_json(data,decision_model,ids=[3,7])

torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])


'{"3": {"embeddings_state0": [0.28, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.0, 0.61, 0.0, 1.05, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19, 0.07, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34, 0.0, 0.0, 0.0, 0.87, 0.0, 0.47, 0.04, 0.0, 0.0, 0.54, 0.0, 0.96, 0.0, 0.0, 0.0, 0.0, 0.48, 0.96, 0.54, 0.0, 0.12, 0.38, 0.0, 0.03, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16, 0.0, 0.0, 0.0, 0.0, 0.0, 0.88, 0.0, 0.0, 0.0, 0.0, 0.36, 0.0, 0.0, 0.0, 0.0, 0.0, 0.52, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.58, 0.0, 0.0, 0.44, 0.0, 0.55, 0.0, 0.39, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.61, 0.0, 0.0, 0.0, 1.28, 0.4, 0.0, 0.0, 0.71, 0.0, 0.41, 0.27, 0.0, 0.0, 0.29, 0.0, 0.0, 0.89, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.78, 0.56, 0.0, 0.36, 0.76, 0.0, 1.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23, 0.0, 0.09, 0.0, 0.23, 0.0, 1.16, 0.53, 0.0, 0.0, 0.0, 0.0, 0.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.15, 0.0, 0.16, 0.0, 0.0, 0.0, 0.0, 0.78, 0.

In [31]:
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt
# import seaborn as sns

# def plot_embedding(dataset,dmodel,decision=0,ax=None,use_optimal=False):
#     if ax is None:
#         fig, ax = plt.subplots(1,1,figsize=(10,10))
#     embeddings, decisions_optimal, decisions_imitation,inputs = get_embeddings(dataset,dmodel,states=[decision])
#     pca = PCA(2)
#     coords = pca.fit_transform(embeddings[0])
#     marks = data.get_state('decision'+ str(decision+1))
#     if use_optimal:
#         predicted = (decisions_optimal > .5).ravel().astype(int)
#     else:
#         predicted = (decisions_imitation > .5).ravel().astype(int)
#     size = [400 for i in marks]
#     sns.scatterplot(data=coords,
#                     x=coords[:,0],
#                     y=coords[:,1],
#                     style=marks,
#                     hue=predicted,
#                     ax=ax,
#                     palette='colorblind',
#                     sizes=size,
#                    )
# plot_embedding(data,decision_model)

In [32]:
# fig,axes = plt.subplots(2,3,figsize=(30,20))
# for ii,opt in enumerate([True,False]):
#     for i in range(3):
#         plot_embedding(data,decision_model,use_optimal=opt,decision=i,ax=axes[ii,i])

In [33]:
def get_default_input(dataset,state=0,ids=None):
    output = get_decision_input(dataset,state=state,ids=ids)
    output = [o.median().to_dict() for o in output]
    output = [{k: (0 if '_ipsi' in k or '_contra' in k else v) for k,v in output[0].items()}]
    return output

get_default_input(data)

[{'1A_contra': 0,
  '1A_ipsi': 0,
  '1B_contra': 0,
  '1B_ipsi': 0,
  '2A_contra': 0,
  '2A_ipsi': 0,
  '2B_contra': 0,
  '2B_ipsi': 0,
  '3_contra': 0,
  '3_ipsi': 0,
  '4_contra': 0,
  '4_ipsi': 0,
  '5A_contra': 0,
  '5A_ipsi': 0,
  '5B_contra': 0,
  '5B_ipsi': 0,
  '6_contra': 0,
  '6_ipsi': 0,
  'AJCC_1': 0.0,
  'AJCC_2': 0.0,
  'AJCC_3': 0.0,
  'AJCC_4': 0.0,
  'African American/Black': 0.0,
  'Asian': 0.0,
  'Aspiration rate Pre-therapy': 0.0,
  'DLT_Infection (Pneumonia)': 0.0,
  'DLT_Infection (Pneumonia) 2': 0.0,
  'DLT_Nephrological': 0.0,
  'DLT_Nephrological 2': 0.0,
  'DLT_Vascular': 0.0,
  'DLT_Vascular 2': 0.0,
  'Hispanic/Latino': 0.0,
  'N-category_0': 0.0,
  'N-category_1': 0.0,
  'N-category_2': 0.0,
  'N-category_3': 0.0,
  'Pathological Grade_0': 0.0,
  'Pathological Grade_1': 0.0,
  'Pathological Grade_2': 0.0,
  'Pathological Grade_3': 1.0,
  'Pathological Grade_4': 0.0,
  'RPLN_contra': 0,
  'RPLN_ipsi': 0,
  'T-category_1': 0.0,
  'T-category_2': 0.0,
  'T-cat

In [34]:
def format_patient(dataset,input_dict,zero_transitions=True):
    #converts patient input features into data input type
    baselines = dataset.processed_df.median().to_dict()
    #set all basline transition states to 0 so my lazy way of checking for fixed values works
    if zero_transitions:
        to_zero = (Const.primary_disease_states 
            + Const.nodal_disease_states 
            + list(Const.modification_types.values()) 
            + list(Const.cc_types.values()))
        for k in to_zero:
            baselines[k] = 0
            baselines[k+' 2'] = 0
    for k,v in input_dict.items():
        baselines[k] = v
    return baselines

format_patient(data,{'ln_cluster_3': 1,'DLT_Neurological': 1})

{'hpv': 1.0,
 'age': 58.155,
 'packs_per_year': 3.0,
 'gender': 1.0,
 'Aspiration rate Pre-therapy': 0.0,
 'total_dose': 70.0,
 'dose_fraction': 2.12,
 'OS (Calculated)': 76.72,
 'Locoregional control (Time)': 74.315,
 'FDM (months)': 75.765,
 'time_to_event': 57.22,
 'Overall Survival (1=alive, 0=dead)': 1.0,
 'LRC': 1.0,
 'DC': 1.0,
 'bilateral': 0.0,
 'White/Caucasion': 1.0,
 'Hispanic/Latino': 0.0,
 'African American/Black': 0.0,
 'Asian': 0.0,
 'cc_none': 0,
 'cc_platinum': 0,
 'cc_cetuximab': 0,
 'cc_others': 0,
 'no_dose_adjustment': 0,
 'dose_modified': 0,
 'dose_delayed': 0,
 'dose_cancelled': 0,
 'dose_delayed_&_modified': 0,
 'regiment_modification': 0,
 'T-category_1': 0.0,
 'T-category_2': 0.0,
 'T-category_3': 0.0,
 'T-category_4': 0.0,
 'N-category_0': 0.0,
 'N-category_1': 0.0,
 'N-category_2': 0.0,
 'N-category_3': 0.0,
 'AJCC_1': 0.0,
 'AJCC_2': 0.0,
 'AJCC_3': 0.0,
 'AJCC_4': 0.0,
 'Pathological Grade_0': 0.0,
 'Pathological Grade_1': 0.0,
 'Pathological Grade_2': 0.

In [35]:
def dict_to_model_input(dataset,fdict,state=0,ttype=torch.FloatTensor,concat=True,zero_transition_states=True):
    fdict = format_patient(dataset,fdict)
    order = get_inputkey_order(dataset,state=state)
    inputs = [torch.tensor([fdict[k] for k in ordersubset]).type(ttype).view(1,-1) for ordersubset in order]
    
    #this is assuming the order is baseline, dlt1, dlt2, primary progression, nodal progression, cc type, dose modification
    def zeroinput(position):
        return torch.zeros(inputs[position].shape).type(ttype)
    if zero_transition_states:
        if state == 0 or state == 1:
            inputs[2] = zeroinput(2)
            inputs[5] = zeroinput(5)
        if state < 1:
            inputs[1] = zeroinput(1)
            inputs[3] = zeroinput(3)
            inputs[4] = zeroinput(4)
            inputs[6] =zeroinput(6)
    if concat:
        inputs = torch.cat(inputs,axis=1)
    #currently at this line its baseline, dlt1, dlt2, pd, nd, cc, modifications
    return inputs
# decision_model(*dict_to_model_input(data,data.processed_df.iloc[7].to_dict(),state=0),position=0)
dict_to_model_input(data,data.processed_df.loc[5].to_dict(),state=1,concat=True,zero_transition_states=False)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,
         20.9500,  0.0000,  1.8000,  1.0000,  0.0000, 38.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000, 72.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [36]:
# def input_df(dataset,state=0):
#     df = dataset.processed_df.copy()
#     order = get_inputkey_order(dataset,state=state)
#     inputs = [df[o] for o in order]
#     df = pd.concat(inputs,axis=1)
# input_df(data)

In [37]:
dict_to_model_input(data,data.processed_df.loc[5].to_dict(),state=1,concat=True,zero_transition_states=True)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  1.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,
         20.9500,  0.0000,  1.8000,  1.0000,  0.0000, 38.0000,  1.0000,  0.0000,
          0.0000,  0.0000,  0.0000, 72.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [38]:
def calculateMahalanobis(y=None, data=None, cov=None):
  
    y_mu = y - np.mean(data)
    if not cov:
        cov = np.cov(data.T)
    inv_covmat = np.linalg.pinv(cov)
    left = np.dot(y_mu, inv_covmat)
    mahal = np.dot(left, y_mu.T)
    return mahal.diagonal()

def get_neighbors_and_embedding(pdata,dataset,decisionmodel,embedding_df=None,state=2,max_neighbors=10,pcas=None):
    decisionmodel.eval()
    if embedding_df is None:
        embedding_df = get_embedding_df(dataset,decisionmodel)
    embeddings = np.stack(embedding_df['embeddings_state'+str(state)].values)
    
    cat = lambda x: torch.cat(x,axis=1)
    
    inputs = dict_to_model_input(dataset,pdata,state=state,zero_transition_states=False).to(decisionmodel.get_device())
    
    
    embedding = decisionmodel.get_embedding(inputs,position=state,use_saved_memory=True)[0].view(1,-1).cpu().detach().numpy()

    mDist = calculateMahalanobis(embedding,embeddings)
    dists = cdist(embedding,embeddings).ravel()
    
    max_neighbors = min(len(dists),max_neighbors)
    min_positions = np.argsort(dists)[:max_neighbors]
    neighbor_ids = dataset.processed_df.index.values[min_positions]
    min_dists = dists[min_positions]
    similarities = 1/(1+min_dists)
    # similarities /= similarities.max() #adjust for rounding errors, self sim should be the max
    if pcas is not None:
        pPca = pcas[state].transform(embedding)[0]
        return neighbor_ids, similarities,embedding[0],pPca, mDist[0]
    return neighbor_ids, similarities, embedding[0], mDist[0]

for state in [0,1,2]:
    print(get_neighbors_and_embedding(test_patient,data,decision_model,state=state)[-1])

torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
4467690121.803214
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
25093981.87168319
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
22948642.370797876


In [39]:
def test_mahalanobis_distances(dataset=None,decision_model=None,state=1,embedding_df=None):
    if embedding_df is None:
        embedding_df = get_embedding_df(dataset,decision_model)
    embeddings = np.stack(embedding_df['embeddings_state'+str(state)].values)
    dists =calculateMahalanobis(embeddings,embeddings) 
    return np.array(dists)

def get_mdist_outlier_threshold(dataset=None,decision_model=None,embedding_df=None):
    return [m.mean() + 2*m.std() for m in [test_mahalanobis_distances(dataset,decision_model,s,embedding_df) for s in [0,1,2]]]

get_mdist_outlier_threshold(data,decision_model)

torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])


[4482766483.702977, 25554765.7824055, 22151186.210548468]

In [40]:
[test_mahalanobis_distances(data,decision_model,s,embedding_df) for s in [0,1,2]]

[array([4.48251408e+09, 4.48242751e+09, 4.48251394e+09, 4.48251384e+09,
        4.48247881e+09, 4.48300997e+09, 4.48239666e+09, 4.48249018e+09,
        4.48254010e+09, 4.48253659e+09, 4.48259716e+09, 4.48249894e+09,
        4.48247624e+09, 4.48231756e+09, 4.48249712e+09, 4.48245912e+09,
        4.48255362e+09, 4.48245595e+09, 4.48247782e+09, 4.48251034e+09,
        4.48190730e+09, 4.48249038e+09, 4.48249884e+09, 4.48248053e+09,
        4.48248426e+09, 4.48252651e+09, 4.48257471e+09, 4.48283273e+09,
        4.48250854e+09, 4.48249348e+09, 4.48249858e+09, 4.48249767e+09,
        4.48251721e+09, 4.48245557e+09, 4.48249279e+09, 4.48245415e+09,
        4.48243367e+09, 4.48250922e+09, 4.48249841e+09, 4.48250012e+09,
        4.48249710e+09, 4.48249891e+09, 4.48250190e+09, 4.48248290e+09,
        4.48249657e+09, 4.48240869e+09, 4.48240992e+09, 4.48252387e+09,
        4.48251603e+09, 4.48253339e+09, 4.48249901e+09, 4.48240337e+09,
        4.48240398e+09, 4.48249836e+09, 4.48243531e+09, 4.482428

In [43]:
def dictify(keys,values):
    return {k:v for k,v in zip(keys,values)}

def get_neighbors_and_embeddings_from_sim(embeddings,dataset,decisionmodel,
                                         embedding_df=None,max_neighbors=100,
                                          pcas=None,
                                          pca_components=2,
                                         ):
    #this is get_embeddings_and_neighbors, but uses the optimal model from get_stuff_For_patient embeddings
    #instead of just kinda not simulation anyhting. adds 1 second on my UIC workstation to the simulation
    if embedding_df is None:
        embedding_df = get_embedding_df(dataset,decisionmodel)
    cat = lambda x: torch.cat(x,axis=1)
    
    embed_arrays = [np.stack(embedding_df['embeddings_state'+str(s)].values) for s in embeddings.keys()]
    if pcas is None:
        pcas = [PCA(pca_components,whiten=True).fit(e) for e in embed_arrays]
    i = 0
    results = {}
    for state, embedding in embeddings.items():
        embedding_array = embed_arrays[i]
        mdist = calculateMahalanobis(embedding.reshape(1,-1),embedding_array)
        
        dists = cdist(embedding,embedding_array).ravel()
        max_neighbors = min(len(dists),max_neighbors)
        min_positions = np.argsort(dists)[:max_neighbors]
        neighbor_ids = dataset.processed_df.index.values[min_positions]
        min_dists = dists[min_positions]
        similarities = 1/(1+min_dists)
        # similarities /= similarities.max() #adjust for rounding errors, self sim should be the max
        pPca = pcas[i].transform(embedding)[0]
        entry = {
            'neighbors': neighbor_ids, 
            'similarities': similarities,
            'embedding': embedding[0],
            'pca': pPca, 
            'mahalanobisDistance': mdist[0]
        }
        results[state] = entry
        i+=1
        
    return results

def get_default_model_inputs(dataset):
    res = []
    for state in [0,1,2]:
        xin = get_default_input(dataset,state)[0]
        xin = dict_to_model_input(dataset,xin,state)
        res.append(xin)
    return res
get_default_model_inputs(data)

[tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
          58.1550,  0.0000,  2.1200,  1.0000,  1.0000,  3.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000, 70.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]]),
 tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.

In [54]:
#changed
def get_stuff_for_patient(patient_dict,data,tmodel1,tmodel2,outcomemodel,decisionmodel,survival_model,
                          symptom_model=None,mdasi_data=None,
                          override_outcome_model=True,
                          state=0,
                          model_type='optimal',
                          timepoints = [1,6,12,18,24,30,36,42,48,54,60],
                          **kwargs):
    #this takes a patient dict and returns the results for a full treatment simulation
    #currently if state > 0 it will check if prior transition states are all zero and if not, will input them
    #currently works with categorical, might have to experiment with passing like -1 for fixed "no" with fixed no dlts
    pdata = format_patient(data,patient_dict)
    
    baseline_inputs = dict_to_model_input(data,pdata,state=0,concat=False) 
    
    
    tmodel1.eval()
    tmodel2.eval()
    outcomemodel.eval()
    decisionmodel.eval()
    survival_model.eval()
    device = 'cpu'
    if torch.cuda.is_available():
        device='cuda'
    tmodel1.set_device(device)
    tmodel2.set_device(device)
    outcomemodel.set_device(device)
    decisionmodel.set_device(device)
    survival_model.set_device(device)
    results = {}
    embeddings = {}
    #do a loop for imitation and a loop for optimal decision making, mod = 3 is imitation
    format_transition = lambda x: x.view(1,-1).to(device)
    #inputs are order baseline, dlt1, dlt2, pd, nd, cc type, dose modifications
    #model output is nx6 -> optimal 1 , 2, 3, imitation 1, 2, 3
    cat = lambda x: torch.cat([xx.to(device) for xx in x],axis=1).to(device)
    
    size_dict = decisionmodel.input_sizes
    
    #baseline, dlt1, dlt2, pd, nd, cc, mod
    input_keys = get_inputkey_order(data)
    
    defaults = get_default_model_inputs(data)
    defaults = [d.to(device) for d in defaults]
    def get_attention(xx, position, offset):
        attention = decisionmodel.get_attributions(xx,target=position+offset, position=position,base=defaults[position])[0].cpu().detach().numpy()
        attention_dict = {
            'step': position,
            'model': 'optimal' if offset == 0 else 'imitation',
            'range': [float(attention.min()),float(attention.max())],
            'baseline': dictify(input_keys[0],attention[0:size_dict['baseline']]),
        }
        pos = size_dict['baseline']
        attention_dict['dlt1'] = dictify(input_keys[1],attention[pos:pos+size_dict['dlt']])
        pos += size_dict['dlt']
        attention_dict['dlt2'] = dictify(input_keys[2], attention[pos:pos+size_dict['dlt']])
        pos += size_dict['dlt']
        attention_dict['pd'] = dictify(input_keys[3], attention[pos:pos+size_dict['pd']])
        pos += size_dict['pd']
        attention_dict['nd'] = dictify(input_keys[4], attention[pos:pos+size_dict['nd']])
        pos += size_dict['nd']
        attention_dict['cc'] = dictify(input_keys[5], attention[pos:pos+size_dict['cc']])
        pos += size_dict['cc']
        attention_dict['modifications'] = dictify(input_keys[6], attention[pos:])
        return attention_dict
        
    memory = get_decision_input(data,state=2)
    memory = cat([df_to_torch(f) for f in memory])
    o1 = decisionmodel(cat(baseline_inputs),position=0)[0]
    
    thresh = lambda x: torch.gt(x,.5).type(torch.FloatTensor)
    
    modifiers = [3] if model_type == 'imitation' else [0]
    if model_type == 'both':
        modifiers = [0,3]
        
    def get_fixed_transitions():
        
        [base, dlt1,_,pd1,nd1,_,mod] =dict_to_model_input(data,pdata,state=1,
                                                          concat=False,zero_transition_states=False)
        [base, _,dlt2,pd2,nd2,cc,_] =dict_to_model_input(data,pdata,state=2,
                                                          concat=False,zero_transition_states=False)
        isfixed = lambda d: not (torch.sum(d) < .00001)
        results = {
            'dlt1': isfixed(dlt1),
            'dlt2': isfixed(dlt2),
            'pd1': isfixed(pd1),
            'pd2': isfixed(pd2),
            'nd1': isfixed(nd2),
            'nd2': isfixed(nd2),
            'cc': isfixed(cc),
            'mod': isfixed(mod)
        }
        return results
    fixed_transitions = get_fixed_transitions()

    def run_simulation(modifier,decision1=None,decision2=None,decision3=None):
        #do this to track malahanobis distances?
        is_default = (modifier == modifiers[0] and decision1 is None and decision2 is None and decision3 is None)
        if is_default:
            embeddings[0] = decisionmodel.get_embedding(cat(baseline_inputs),position=0,
                                                                  use_saved_memory=True)
            
        #transition 1 model uses usebaline + decision
        if decision1 is not None:
            d1 = torch.tensor([[decision1]]).type(torch.FloatTensor)
            d1_attention=0
        else:
            d1 = o1[0+modifier].view(1,-1)
            d1_attention = get_attention(cat(baseline_inputs),0,modifier)
            
        propensity1 = o1[0+3].view(1,-1)
            
        tinput1 = cat([baseline_inputs[0],thresh(d1)])
        
        ytransition = tmodel1(tinput1)
        [ypd1,ynd1,ymod,ydlt1] = ytransition['predictions']

        [ypd1, ynd1, ymod] = [format_transition(i) for i in [ypd1,ynd1,ymod]]
        
        #I try to make this work in the model but it just thinks there's no outcome and softmaxes them all often
        d1_thresh = torch.gt(d1,.5).view(-1,1).to(device)
        ypd1[:,0:2] = ypd1[:,0:2]*d1_thresh
        ynd1[:,0:2] = ynd1[:,0:2]*d1_thresh
        
        oinput2 = dict_to_model_input(data,pdata,state=1,concat=False,zero_transition_states=False)
        #if the input stuff has a value for transition states and state passed is > 0, fix them
        
        #check if I should actually use the transition states
        if state > 0 and fixed_transitions['dlt1']:
            ydlt1 = torch.clone(oinput2[1])
        else:
            oinput2[1] = ydlt1.view(1,-1)
        if state > 0 and fixed_transitions['pd1']:
            ypd1 = torch.clone(oinput2[3])
        else:
            oinput2[3] = ypd1
        if state > 0 and fixed_transitions['nd1']:
            ynd1 = torch.clone(oinput2[4])
        else:
            oinput2[4] = ynd1
        if state > 0 and fixed_transitions['mod']:
            ymod = torch.clone(oinput2[6])
        else:
            oinput2[6] = torch.clone(ymod)
            
        d2_full = decisionmodel(cat(oinput2),position=1)
        if decision2 is not None:
            d2 = torch.tensor([[decision2]]).type(torch.FloatTensor)
            d2_attention=0
        else:
            d2 = d2_full[0,1+modifier].view(1,-1)
            d2_attention = get_attention(cat(oinput2),1,modifier)
            
        propensity2 = d2_full[0,4].view(1,-1)
            
        if is_default:
            embeddings[1] = decisionmodel.get_embedding(cat(oinput2),position=1,use_saved_memory=True)
            
        #transition 2 modle uses baseline + pd1 + nd1 + modification + dlt1 + decision 1 + deicsion 2
        tinput2 = [baseline_inputs[0], ypd1, ynd1, ymod,ydlt1, thresh(d1),thresh(d2)]

        tinput2 = cat(tinput2)
        
        ytransition2 = tmodel2(tinput2)
        [ypd2, ynd2, ycc, ydlt2] = ytransition2['predictions']
        [ypd2, ynd2, ycc] = [format_transition(i) for i in [ypd2,ynd2,ycc]]
        
        oinput3 = oinput2[:]
        #check if I should use the transition states again
        if state > 1 and fixed_transitions['dlt2']:
            ydlt2 = torch.clone(oinput3[2])
        else:
            oinput3[2] = ydlt2.view(1,-1)
        if state > 1 and fixed_transitions['pd2']:
            ypd2 = torch.clone(oinput3[3])
        else:
            oinput3[3] = ypd2
        if state > 1 and fixed_transitions['nd2']:
            ynd2 = torch.clone(oinput3[4])
        else:
            oinput3[4] = ynd2
        if state > 1 and fixed_transitions['cc']:
            ycc = torch.clone(oinput3[5])
        else:
            oinput3[5] = torch.clone(ycc)

        d3_full = decisionmodel(cat(oinput3),position = 2)
        if decision3 is not None:
            d3 = torch.tensor([[decision3]]).type(torch.FloatTensor)
            d3_attention=0
        else:
            d3 = d3_full[0,2+modifier].view(1,-1)
            d3_attention = get_attention(cat(oinput3),2,modifier)
        propensity3= d3_full[0,4].view(1,-1)
        
        if is_default:
            embeddings[2] = decisionmodel.get_embedding(cat(oinput3),position=2,use_saved_memory=True)
        #outcomes uses baseline + pd2 + nd2 + cc type + dlt2 + decision 1,2,3
        tinput3 = [baseline_inputs[0], ypd2, ynd2, ycc, ydlt2, thresh(d1), thresh(d2), thresh(d3)]
        tinput3 = cat(tinput3)
        outcomes = outcomemodel(tinput3)
        
        entry = {
            'decision1': d1.cpu().detach().numpy()[0][0],
            'decision2': d2.cpu().detach().numpy()[0][0],
            'decision3': d3.cpu().detach().numpy()[0][0],
            'decision1_attention': d1_attention,
            'decision2_attention': d2_attention,
            'decision3_attention': d3_attention,
            'propensity1': propensity1.cpu().detach().numpy()[0][0],
            'propensity2': propensity2.cpu().detach().numpy()[0][0],
            'propensity3': propensity3.cpu().detach().numpy()[0][0],
        }
        
        def add_to_entry(tmodel_output,names):
            pred = tmodel_output['predictions']
            lower = tmodel_output['5%']
            upper = tmodel_output['95%']
            for suffix,values in zip(['','_5%','_95%'],[pred,lower,upper]):
                for name, v in zip(names,values):
                    v = v.cpu().detach().numpy()
                    if name != 'outcomes':
                        v = v[0]
                    #because of softmax the model will output 33% for pd and nd with no ic when it should be fixed to 0
                    if entry['decision1'] < .5 and ('pd1' in name or 'nd1' in name):
                        v = np.zeros(v.shape)
                    entry[name+suffix] = v
        add_to_entry(ytransition,['pd1','nd1','modifications','dlt1'])
        add_to_entry(ytransition2,['pd2','nd2','cc_type','dlt2'])
        add_to_entry(outcomes,['outcomes'])
        
        #add time to event  for each event, each is a seperate entry unlike the grouped outcomes
        tte = survival_model.time_to_event(tinput3,n_samples=1)
        tte_order = survival_model.outcome_names
        for v, name in zip(tte,tte_order):
            entry[name] = v.cpu().detach().numpy()[0]
        #tte = survival_model.time_to_event(tinput3)
        # add_to_entry(tte,tte_order)
        
        scurve_entry = {}
        survival_curves = survival_model(tinput3,timepoints)
        for name,scurve in zip(tte_order,survival_curves):
            scurve = [np.round(v.cpu().detach().numpy()[0],3) for v in scurve]
            scurve_entry[name] = scurve
        scurve_entry['times'] = timepoints
        entry['survival_curves'] = scurve_entry
        #put the 4year probability for the timeseries input in case I want to override the other model
        fouryear_probs = survival_model(tinput3,48)
        for name, probs in zip(tte_order,fouryear_probs):
            probs = probs[0].detach().cpu().numpy()[0]
            entry[name+'(4yr)'] = np.round(probs,3)
        key = 'optimal' if modifier < 1 else 'imitation'
        if decision1 is not None:
            key += '_decision1-'+str(decision1)
        if decision2 is not None:
            key += '_decision2-'+str(decision2)
        if decision3 is not None:
            key += '_decision3-'+str(decision3)
        results[key] = entry

    
    with torch.no_grad():
        for modifier in modifiers:
            for d1_fixed in [None,0,1]:
                for d2_fixed in [None,0,1]:
                    for d3_fixed in [None,0,1]:
                        #we only need to do all fixed outcomes once
                        if d1_fixed is not None and d2_fixed is not None and d3_fixed is not None and modifier != modifiers[0]:
                            continue
                        run_simulation(modifier,d1_fixed,d2_fixed,d3_fixed)

    for k,v in embeddings.items():
        embeddings[k] = v.cpu().detach().numpy()
    embedding_results = get_neighbors_and_embeddings_from_sim(embeddings,data,decisionmodel,**kwargs)
    
    res = {'simulation': results,'embeddings': embedding_results}
    
    symptoms = {}
    if symptom_model is not None and mdasi_data is not None:
        symptoms = get_knn_predictions(pdata,symptom_model,mdasi_data,symptom_subset=['choke','drymouth'])
        res['symptoms'] = symptoms
    return res

from time import time
t1 = time()
test_patient= get_test_patient(data,7,False)
test_results = get_stuff_for_patient(test_patient,data,
                                     transition_model1,
                                     transition_model2,
                                     outcome_model,
                                     decision_model,
                                     survival_model,
#                                      symptom_model=sp,
#                                      mdasi_data=mdasi,
                                     state=1,
                                     max_neighbors=2,
                                     model_type='imitation',
                                    )
t2 = time()
print(t2-t1)
test_results

torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
6.819299936294556


{'simulation': {'imitation': {'decision1': 0.995538,
   'decision2': 0.013968321,
   'decision3': 0.43196642,
   'decision1_attention': {'step': 0,
    'model': 'imitation',
    'range': [-0.004109903455914532, 0.004785101147933047],
    'baseline': {'1A_contra': -0.0,
     '1A_ipsi': 0.0,
     '1B_contra': 0.003485457556594306,
     '1B_ipsi': -0.0,
     '2A_contra': 0.0,
     '2A_ipsi': -0.0,
     '2B_contra': -0.0,
     '2B_ipsi': 0.0,
     '3_contra': -0.0,
     '3_ipsi': 0.0,
     '4_contra': -0.0,
     '4_ipsi': -0.0,
     '5A_contra': 0.0,
     '5A_ipsi': 0.0,
     '5B_contra': 0.0,
     '5B_ipsi': 0.0,
     '6_contra': -0.0,
     '6_ipsi': 0.0,
     'AJCC_1': 0.0,
     'AJCC_2': 0.004785101147933047,
     'AJCC_3': -0.0,
     'AJCC_4': 0.0,
     'African American/Black': -0.0,
     'Asian': 0.0,
     'Aspiration rate Pre-therapy': -0.0,
     'DLT_Infection (Pneumonia)': -0.0,
     'DLT_Infection (Pneumonia) 2': -0.0,
     'DLT_Nephrological': -0.0,
     'DLT_Nephrological 2': 0

In [55]:
t1 = time()
test_patient= get_test_patient(data,7,False)
test_results = get_stuff_for_patient(test_patient,data,
                                     transition_model1,
                                     transition_model2,
                                     outcome_model,
                                     decision_model,
                                     survival_model,
                                     symptom_model=sp,
                                     mdasi_data=mdasi,
                                     state=1,
                                     max_neighbors=2,
                                     model_type='imitation',
                                    )
t2 = time()
print(t2-t1)
test_results

torch.Size([536, 86])
torch.Size([536, 86])
torch.Size([536, 86])
6.908932447433472


{'simulation': {'imitation': {'decision1': 0.995538,
   'decision2': 0.013968321,
   'decision3': 0.43196642,
   'decision1_attention': {'step': 0,
    'model': 'imitation',
    'range': [-0.004109903455914532, 0.004785101147933047],
    'baseline': {'1A_contra': -0.0,
     '1A_ipsi': 0.0,
     '1B_contra': 0.003485457556594306,
     '1B_ipsi': -0.0,
     '2A_contra': 0.0,
     '2A_ipsi': -0.0,
     '2B_contra': -0.0,
     '2B_ipsi': 0.0,
     '3_contra': -0.0,
     '3_ipsi': 0.0,
     '4_contra': -0.0,
     '4_ipsi': -0.0,
     '5A_contra': 0.0,
     '5A_ipsi': 0.0,
     '5B_contra': 0.0,
     '5B_ipsi': 0.0,
     '6_contra': -0.0,
     '6_ipsi': 0.0,
     'AJCC_1': 0.0,
     'AJCC_2': 0.004785101147933047,
     'AJCC_3': -0.0,
     'AJCC_4': 0.0,
     'African American/Black': -0.0,
     'Asian': 0.0,
     'Aspiration rate Pre-therapy': -0.0,
     'DLT_Infection (Pneumonia)': -0.0,
     'DLT_Infection (Pneumonia) 2': -0.0,
     'DLT_Nephrological': -0.0,
     'DLT_Nephrological 2': 0

In [27]:
# def get_default_predictions(dm):
#     res  = []
#     for state in [0,1,2]:
#         if hasattr(dm,'memory'):
#             mem = dm.memory[state]
#             mem = torch.median(mem,dim=0)[0].type(torch.FloatTensor)
#             val = dm(mem.reshape(1,-1),position=state)
#         else:
#         res.append(val.cpu().detach().numpy())
#     return np.vstack(res)

def get_default_predictions(dm,dataset):
    res = []
    for state in [0,1,2]:
        xin = get_default_input(dataset,state)[0]
        xin = dict_to_model_input(dataset,xin,state).to(dm.get_device())
        val = dm(xin,position=state)
        res.append(val.cpu().detach().numpy())
    return np.vstack(res)

def get_default_prediction_json(dm,dataset):
    vals = get_default_predictions(dm,dataset)
    res={}
    for i,model in enumerate(['optimal','imitation']):
        entry = {}
        for state,decision in enumerate(Const.decisions):
            val = vals[state, state + (3*i)]
            entry[decision] = val
        res[model] = entry
    return res

get_default_prediction_json(decision_model,data)

{'optimal': {'Decision 1 (Induction Chemo) Y/N': 0.038047604,
  'Decision 2 (CC / RT alone)': 0.046430662,
  'Decision 3 Neck Dissection (Y/N)': 0.90782195},
 'imitation': {'Decision 1 (Induction Chemo) Y/N': 0.77922934,
  'Decision 2 (CC / RT alone)': 0.18129703,
  'Decision 3 Neck Dissection (Y/N)': 0.23397787}}

In [ ]:
import datetime
def np_converter(obj):
    #converts stuff to vanilla python  for json since it gives an error with np.int64 and arrays
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.float32):
        return np.round(float(obj),3)
    elif isinstance(obj, float):
        return round(float(obj),3)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, datetime.datetime) or isinstance(obj, datetime.time):
        return obj.__str__()
    print('np_converter cant encode obj of type', obj,type(obj))
    return obj

import simplejson
keys = ['outcomes','pd1','nd1','decision1','decision1_attention','decision2_attention']
simplejson.dumps(test_results,default=np_converter)

In [ ]:
type(test_results['optimal']['decision1'])

In [ ]:
test_results['optimal'].keys()

In [ ]:
###PseudoCode for treating the patient
def getGroup(isTreated,candidates, new_patient_propensity, minimum_group_size,caliper_distance_base,scale_increment):
    ##gets patients in the treated or untreated group that have a propensity similar to new_patient_propensity
    group = []
    caliper_distance_scale = scale_increment
    #gradually increase caliper size until we have minimum_group_size patients or we run out of candidates
    while len(group) < minimum_group_size and len(candidates) > 1:
        #update caliper distance
        caliper_distance = caliper_distance_base*caliper_distance_scale
        for patient in candidates:
            #check if the patient is in the treated group and has a close enough similarity score
            if patient.treatment == isTreated and absolute_value(new_patient_propensity - patient.propensity) < caliper_distance:
                #avoid repeating patients in subsequent loops
                delete patient from candidates
                #add patient to group if they are valid
                group.push(patient)
        #update caliper distance by scale_increment %
        caliper_distance_scale = caliper_distance_scale*scale_increment
    return group
        
def getNeighbors(cohort,new_patient_propensity, similiarity_filter_size, minimum_group_size, caliper_distance_scale, scale_increment):
    #filter out the top patients by similarity
    cohort = sort(cohort, key = lambda paient: patient.similarity_with_new_patient)
    cohort = cohort[0:similarity_filter_size]
    
    #calculate the standard deviation of the logit of the propensity scores of the cohort
    cohort_propensities = [patient.propensity for patient in cohort]
    caliper_distance_base = standard_deviation(logit(cohort_propensities))
    caliper_distance_scale = .1
    
    #calculate treated and untreated groups with scaling propensity independently
    treated_group = getGroup(True, copy(cohort), *args)
    untreated_group = getGroup(False, copy(cohort), *args)
    return treated_group, untreated_group

def localAverageTreatementEffect(outcome, *args):
    treated_group, untreated_group = getNeighbors(*args)
    treated_average = mean([patient[outcome] for patient in treated_group])
    untreated_average = mean([patient[outcome] for patient in untreated_group])
    return treated_average - untreated_average